In [1]:
import torch
from torch.utils import data
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
# CPU 혹은 GPU 장치 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

In [5]:
trainset = datasets.MNIST(
    root = "./data/",
    train= True,
    download = True,
    transform = transform)

testset = datasets.MNIST(
    root = "./data/",
    train = False,
    download = True,
    transform = transform)

In [6]:
batch_size = 32

In [7]:
# 데이터 공급 객체 선언
train_loader = data.DataLoader(dataset = trainset, batch_size = batch_size)
test_loader = data.DataLoader(dataset = testset, batch_size = batch_size)

In [8]:
class Net(nn.Module):
    def __init__(self):
        '''모델 구조 정의'''
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 10)
        
    def forward(self, x):
        # 입력 데이터 = x
        # x.shape([배치사이즈, 채널, 높이, 너비])
        x = x.view(-1, 28 * 28)
        
        # x.shape([배치사이즈, 784])
        x = F.relu(self.fc1(x))
        
        # x.shape([배치사이즈, 앞 레이어의 출력 개수])
        x = self.fc2(x)
        
        return x

In [9]:
# 모델 객체 선언
model = Net().to(device)

In [10]:
# 옵티마이저 설정
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [11]:
# 학습 함수 정의
def train(model, train_loader, optimizer):
    # 모델을 학습 모드로 전환
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 device 로 보냄
        data, target = data.to(device), target.to(device)
        
        # 매 이터레이션 마다 기울기를 계산하기 위해 zero_grad() 호출
        optimizer.zero_grad()
        
        # 실제 모델의 예측값 받아오기
        output = model(data)
        
        # 정답 데이터와의 cross entropy Loss 계산
        # Loss는 mini batch의 클래스의 오차 평균값
        loss = F.cross_entropy(output, target)
        
        # 기울기 계산
        loss.backward()
        
        # 가중치 수정
        optimizer.step()

In [12]:
def evaluate(model, test_loader):
    # 모델을 평가모드로 전환
    model.eval()
    
    # 필요한 변수 초기화
    # test 과정에서의 Loss
    # 실제 모델의 예측이 정답과 맞은 횟수(correct)
    test_loss = 0
    correct = 0
    
    with torch.no_grad(): # 평가 시에는 기울기를 계산하지 않으므로, no_grad를 명시
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            # 모든 오차 계산하기
            test_loss += F.cross_entropy(output, target, reduction = "sum").item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측
            # 예측과 정답을 비교하여 일치하면 correct에 1 더하기
            pred = output.max(1, keepdim = True)[1]
            
            # eq() : 값이 일치하면 1, 아니면 0
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    # 정확도 계산
    test_accuracy = 100 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [13]:
for epoch in range(1, 6):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f"[{epoch}] Test Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.2f}%")

[1] Test Loss : 0.4094, Accuracy : 89.04%
[2] Test Loss : 0.3283, Accuracy : 90.72%
[3] Test Loss : 0.2953, Accuracy : 91.66%
[4] Test Loss : 0.2724, Accuracy : 92.27%
[5] Test Loss : 0.2537, Accuracy : 92.80%
